In [1]:
import os
import pickle
import pandas as pd 
import numpy as np
import matplotlib.pyplot  as plt
import seaborn as sn
%matplotlib inline
import pandas as pd
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("/home/growlt199/Downloads/manufacturing_project/data/flight_processed_data_wo_stops.csv",index_col=0)

In [3]:
df.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price,travel_route
0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953,Delhi-Mumbai
1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953,Delhi-Mumbai
2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956,Delhi-Mumbai
3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955,Delhi-Mumbai
4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955,Delhi-Mumbai


In [4]:
#removing colums which i dont want to use in model 
df.drop(['flight','travel_route'],axis=1,inplace=True)

In [5]:
df.head()

,airline,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,SpiceJet,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,SpiceJet,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,AirAsia,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,Vistara,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,Vistara,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955


In [6]:
def data_to_one_hot(df):
    # get dummy variables for multiple columns
    dummy_cols = ['airline', 'departure_time','arrival_time','source_city','destination_city','stops','class']
    df1_dummy = pd.get_dummies(df[dummy_cols])

    # concatenate original DataFrame with dummy variable DataFrame
    df1_concat = pd.concat([df.drop(dummy_cols, axis=1), df1_dummy], axis=1)

    # display result
    return df1_concat

In [7]:
def data_label_encoding(df):
    from sklearn.preprocessing import LabelEncoder

    # Create an instance of LabelEncoder
    label_encoder = LabelEncoder()

    # Specify the list of categorical columns to encode
    categorical_columns = ['airline', 'departure_time','arrival_time','source_city','destination_city','stops','class']

    # Apply label encoding to each categorical column
    for column in categorical_columns:
        df[column] = label_encoder.fit_transform(df[column])
    
    return df

In [8]:
def spilt_data(train_data,test_data):
    #create x and y where x are features for the model and y as target
    x_train = train_data.drop('price',axis=1)
    y_train = train_data['price']

    #for test data, create x and y where x are features for the model and y as target
    x_test = test_data.drop('price',axis=1)
    y_test = test_data['price']
    
    return x_train,y_train,x_test,y_test

In [9]:
def spilt_inti_x_y(data):
    
    #for test data, create x and y where x are features for the model and y as target
    x = test_data.drop('price',axis=1)
    y = test_data['price']
    return x,y

In [10]:
def divide_into_train_test(data):
    # Split the data into training and testing sets
    train_data, test_data = train_test_split(data, test_size=0.2, random_state=123)

    # Print the shapes of the train and test sets
    #print("Training set shape:", train_data.shape)
    #print("Testing set shape:", test_data.shape)
    
    return train_data,test_data

In [11]:
def model_random_forest(x_train,y_train,x_test,y_test):

    import time

    start_time = time.time()
    
    #model 
    regr = RandomForestRegressor(n_estimators=500,
        max_depth=None,
        min_samples_split=5,
        min_samples_leaf=1,
        max_features='sqrt',
        random_state=123)

    #training model input as x_train and y_tarin
    regr.fit(x_train,y_train)

#     print('score on the testing data :-',regr.score(x_test,y_test))
#     print('score on the training data :-',regr.score(x_train,y_train))

    #model is creted show give x test data as input and retuen predictd data 
    predictions = regr.predict(x_test)

    #MAE
    print('MAE:-',metrics.mean_absolute_error(y_test,predictions))

    #MSE
    print('MSE:-',metrics.mean_squared_error(y_test,predictions))

    #RMSE
    print('RMSE:-',np.sqrt(metrics.mean_squared_error(y_test,predictions)))

    #r2
    print("r2",r2_score(y_test, predictions))

    # Code or function to measure execution time
    end_time = time.time()
    execution_time = end_time - start_time
    print("Execution time:", execution_time, "seconds")
    
    return regr,predictions

In [12]:
def model_into_pickle(model, filename):

    # Create the directory if it doesn't exist
    directory = os.path.dirname(filename)
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Save the model to the specified filename
    with open(filename, 'wb') as file:
        pickle.dump(model, file)

In [13]:
def predict_with_model(data, model_path):
    # Load the model from the pickle file
    with open(model_path, 'rb') as file:
        model = pickle.load(file)

    # Make predictions on the new data
    predictions = model.predict(data)
    
    return predictions

In [14]:
def testing_new_data(data,filename_1,filename_2,filename_3):
    
    #convert into one hot and label sncoding 
    data_1 = data_to_one_hot(data)
    data_2 = data_label_encoding(data)
    
    # Make predictions on the new data
    predictions_1 = predict_with_model(data_1, filename_1)
    predictions_2 = predict_with_model(data_2, filename_2)
    
    # Stack the predictions horizontally
    stacked_predictions = np.column_stack((predictions_1, predictions_2))

    # Combine original features with stacked predictions
    X_stacked = np.column_stack((data_1,data_2,stacked_predictions))
    
    final_prediction = predict_with_model(X_stacked, filename_3)
    
    return predictions_1,predictions_2,final_prediction

In [15]:
#final script

#split data into two part where one is train main part and econd main test part
train_data,test_data = divide_into_train_test(df)

#devide train data into data 1 and data 2 for one hot and klabel encoding
data_1 = data_to_one_hot(train_data)
data_2 = data_label_encoding(train_data)

#divide data1 and data 2 into two part train and test
train_data_1,test_data_1 = divide_into_train_test(data_1)
train_data_2,test_data_2 = divide_into_train_test(data_2)

#divide train test into fetures and target
X_train_1,y_train_1,X_test_1,y_test_1=spilt_data(train_data_1,test_data_1)
X_train_2,y_train_2,X_test_2,y_test_2=spilt_data(train_data_2,test_data_2)

#model train part 
model_1,predictions_1 = model_random_forest(X_train_1,y_train_1,X_test_1,y_test_1)
model_2,predictions_2 = model_random_forest(X_train_2,y_train_2,X_test_2,y_test_2)

#dump in pickle file 
filename_1 = '/home/growlt199/Downloads/manufacturing_project/model_training/model_stacking/model_1_random_forest_regression.pkl'
filename_2 = '/home/growlt199/Downloads/manufacturing_project/model_training/model_stacking/model_2_random_forest_regression.pkl'

model_into_pickle(model_1,filename_1)
model_into_pickle(model_2,filename_2)

# Stack the predictions horizontally
stacked_predictions = np.column_stack((predictions_1, predictions_2))

# Combine original features with stacked predictions
X_stacked = np.column_stack((X_test_1,X_test_2, stacked_predictions))

# Second-level model
meta_model =  RandomForestRegressor(n_estimators=500,
        max_depth=None,
        min_samples_split=5,
        min_samples_leaf=1,
        max_features='sqrt',
        random_state=123)

# Train the second-level model using combined features
meta_model.fit(X_stacked, y_test_1)

#dump into pickle file
filename_3 = '/home/growlt199/Downloads/manufacturing_project/model_training/model_stacking/meta_model_random_forest_regression.pkl'
model_into_pickle(meta_model,filename_3)


MAE:- 1222.4730106394506
MSE:- 7511195.675372415
RMSE:- 2740.6560665965394
r2 0.9854142533815873
Execution time: 69.58808064460754 seconds
MAE:- 1260.295667920896
MSE:- 7632358.251903353
RMSE:- 2762.672302663375
r2 0.9851789717144209
Execution time: 62.588926792144775 seconds


# testing in new data 

In [ ]:
#final test
test_data_x,test_data_y = spilt_inti_x_y(test_data)

#final prediction 
predit_1,prdict_2,final_predictions = testing_new_data(test_data_x,filename_1,filename_2,filename_3)

#MAE
print('MAE:-',metrics.mean_absolute_error(test_data_y,final_predictions))

#MSE
print('MSE:-',metrics.mean_squared_error(test_data_y,final_predictions))

#RMSE
print('RMSE:-',np.sqrt(metrics.mean_squared_error(test_data_y,final_predictions)))

#r2
print("r2",r2_score(test_data_y, final_predictions))

In [ ]:
len(test_data_y)

In [ ]:
len(predit_1)

In [ ]:
len(prdict_2)

In [ ]:
len(final_predictions)

In [ ]:
import pandas as pd
import numpy as np

# Assuming you have one dataframe: df
# and three arrays: array1, array2, array3

# Convert arrays to dataframes
array1_df = pd.DataFrame({'predit_1': predit_1})
array2_df = pd.DataFrame({'prdict_2': prdict_2})
array3_df = pd.DataFrame({'final_predictions': final_predictions})

# Merge the dataframe and arrays horizontally (column-wise) without including indices
model_stacking_results= pd.concat([test_data_x['duration','days_left'].reset_index(drop=True),test_data_y.reset_index(drop=True), array1_df, array2_df, array3_df], axis=1)

# Save the merged dataframe to a CSV file
model_stacking_results.to_csv('model_stacking_results.csv', index=False)


In [ ]:
test_data_y

In [ ]:
model_stacking_results